In [ ]:
def process_feeds(feed_urls):
    """
    Process multiple RSS feeds and save metadata for each feed.

    Args:
        feed_urls (list): List of RSS feed URLs.

    Returns:
        None
    """
    proxy_settings = get_proxy_settings()
    app_proxy = proxy_settings["app_proxy"]
    username = proxy_settings["username"]
    password = proxy_settings["password"]
    output_dir = "rss_metadata"
    os.makedirs(output_dir, exist_ok=True)

    for feed_url in feed_urls:
        print(f"Processing feed: {feed_url}")
        try:
            # Retry fetching feed content with backoff
            feed_content = retry_with_backoff(
                fetch_feed_content_with_proxy,
                retries=3,
                backoff_factor=2,
                feed_url=feed_url,
                app_proxy=app_proxy,
                username=username,
                password=password
            )
            if feed_content:
                # Parse and save metadata
                feed_data = parse_feed(feed_content)
                metadata = [
                    {
                        "title": getattr(entry, "title", "No Title"),
                        "url": getattr(entry, "link", "No URL"),
                        "published_date": getattr(entry, "published", "No Date"),
                    }
                    for entry in feed_data.entries
                ]
                output_file = os.path.join(output_dir, f"{feed_url.split('/')[-1]}.json")
                with open(output_file, "w") as f:
                    json.dump(metadata, f, indent=4)
                print(f"Metadata saved to {output_file}")
            else:
                print(f"Failed to fetch feed: {feed_url}")
        except Exception as e:
            print(f"Error processing feed {feed_url}: {e}")


import time

def retry_with_backoff(func, retries=3, backoff_factor=2, *args, **kwargs):
    """
    Retry a function with exponential backoff.

    Args:
        func (callable): Function to call.
        retries (int): Number of retries.
        backoff_factor (int): Factor by which the wait time increases.
        *args, **kwargs: Arguments to pass to the function.

    Returns:
        Any: Return value of the function if successful.

    Raises:
        Exception: The last exception raised if all retries fail.
    """
    for attempt in range(retries):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            if attempt < retries - 1:
                wait_time = backoff_factor ** attempt
                print(f"Retry {attempt + 1}/{retries} failed. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                print(f"All retries failed for {func.__name__}. Error: {e}")
                raise
